In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
from pyspark.sql.functions import split, regexp_replace
from pyspark.sql.functions import explode
from sparkmeasure import StageMetrics


spark = SparkSession.builder.appName('q8').config("spark.jars", "/home/athina/Downloads/spark-measure_2.12-0.17.jar").getOrCreate()
stagemetrics = StageMetrics(spark)

stagemetrics.begin()

df_mv = (spark.read.format('csv').option('header', 'true').option('infereSchema', 'true').load("movie.csv"))

#df_tag = spark.read.format('csv').option('header', 'true').option('infereSchema', 'true').load('/home/administrator/Downloads/movielens/tag.csv')
df_rt = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('rating.csv')


#--Q8 START--------------------------------------------------------------------------------------------------------

#movies_tags = df_tag.join(df_mv, df_tag.movieId == df_mv.movieId, 'inner')
#movies_tags.show()
#>>df_mv.select('genres').orderBy('genres').distinct().show()
#df_mv = (spark.read.format('csv').option('header', 'true').option('infereSchema', 'true').load('movie.csv'))


#used for getting the different genres
gen = df_mv.select('movieId',split(df_mv.genres, '\|').alias('lgenres'))
#>>gen.show()

df_mv_plus = df_mv.join(gen, df_mv.movieId == gen.movieId, 'inner').drop(df_mv.movieId)
#>>df_mv_plus.show()



df_mv_plus_expl = df_mv_plus.withColumn("lgenres", explode(df_mv_plus.lgenres))
df_mv_plus_expl= df_mv_plus_expl.filter(df_mv_plus_expl.lgenres != '(no genres listed)')
#>>df_mv_plus_expl.show()

#df_mv_plus_expl.filter(df_mv_plus_expl.lgenres == "Action").show()
d= df_rt.groupBy('movieId').count()
d=d.withColumnRenamed('count', 'numOfRatings')
num_of_rat_and_lgenres = df_mv_plus_expl.join(d, df_mv_plus_expl.movieId == d.movieId, 'inner').drop(d.movieId)
#>>num_of_rat_and_lgenres.orderBy('numOfRatings', ascending = False).show()

wanted = num_of_rat_and_lgenres.groupBy('lgenres', 'title').max('numOfRatings')#orderBy('numOfRatings', ascending = False).show()
#>>wanted.orderBy('max(numOfRatings)', ascending = False).show()

#wanted.select('title', 'lgenres', 'max(numOfRatings)').groupBy('lgenres').max('max(numOfRatings)').orderBy('lgenres').show()

#q8
#wanted
wanted2 = wanted.select('lgenres', 'max(numOfRatings)').groupBy('lgenres').agg(functions.max("max(numOfRatings)")).orderBy('lgenres')#.show(5)

wanted2 = wanted2.join(wanted, (wanted2['lgenres'] == wanted['lgenres']) & (wanted2['max(max(numOfRatings))'] == wanted['max(numOfRatings)']), 'inner').select(wanted.title, wanted2.lgenres, wanted2['max(max(numOfRatings))']).orderBy('lgenres')
wanted2.show(5)

stagemetrics.end()

#print performance metrics
stagemetrics.print_report()
spark.stop()

#--Q8 END----------------------------------------------------------------------------------------------------------

+--------------------+---------+----------------------+
|               title|  lgenres|max(max(numOfRatings))|
+--------------------+---------+----------------------+
|Jurassic Park (1993)|   Action|                 59715|
|Jurassic Park (1993)|Adventure|                 59715|
|    Toy Story (1995)|Animation|                 49695|
|    Toy Story (1995)| Children|                 49695|
| Pulp Fiction (1994)|   Comedy|                 67310|
+--------------------+---------+----------------------+
only showing top 5 rows


Scheduling mode = FIFO
Spark Context default degree of parallelism = 2
Aggregated Spark stage metrics:
numStages => 11
numTasks => 1016
elapsedTime => 67268 (1.1 min)
stageDuration => 70885 (1.2 min)
executorRunTime => 106221 (1.8 min)
executorCpuTime => 82762 (1.4 min)
executorDeserializeTime => 5833 (6 s)
executorDeserializeCpuTime => 4052 (4 s)
resultSerializationTime => 73 (73 ms)
jvmGCTime => 2936 (3 s)
shuffleFetchWaitTime => 11 (11 ms)
shuffleWriteTime => 123